In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [3]:
import torch
import os
import uuid

from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate

import numpy as np
import matplotlib.pyplot as plt

from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
import transformers

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from transformers import pipeline, AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

/home/choi/anaconda3/envs/RAG_langchain_v02/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 단계 1 : 데이터셋 로드

import json
# JSON 파일에서 데이터를 불러와 Document 객체로 복원
with open("/media/choi/HDD1/mmaction2/data/Korea_construction_standard/LHCS_qna_id.json", "r", encoding="utf-8") as f:
    documents_data = json.load(f)

documents_512 = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in documents_data
]

print(f"문서의 페이지수 : {len(documents_512)}")

문서의 페이지수 : 1255


In [5]:
# 단계 2 : 임베딩 모델 로드
# EM_klue_nli = HuggingFaceEmbeddings(model_name = '/home/choi/Git/ConSRoBERTa/output/2024_2/klue_nli_top_2e_55.24/kor_multi_klue-2024-09-19_15-26-29e1e2')
# EM_open = OpenAIEmbeddings(model = 'text-embedding-3-large') # 최신 GPT4 유로 모델
# EM_klue_vanilla = HuggingFaceEmbeddings(model_name = 'klue/roberta-base')
EM_MNRL_MRL = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')
# embedding_models = [
#     EM_klue_nli,
#     EM_MNRL_MRL,
#     EM_open,
#     EM_klue_vanilla,
# ]

/tmp/ipykernel_16779/3885115005.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  EM_MNRL_MRL = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')


In [6]:
# top 5검색 결과와 답변 생성까지 저장
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.0)

answer_prompt = ChatPromptTemplate.from_template(
    """
- <Question> 이후에 오는 질문은 <Text> 이후에 오는 문서와 관련된 질문이야.
- <Text> 이후의 문서를 참고해서 <Question> 이후에 오는 질문에 대한 답변을 생성해줘.
- 정확하고 간결하게 답변해줘
- 답변은 한글로 출력해줘


<Question> : {question}
<Text>: {input}
----
<Answer>:
    """
)

class AnswerParser(StrOutputParser):
    def parse(self, response):
        # "Question:" 이후의 텍스트를 추출
        if "Answer>:" in response:
            return response.split("Answer>:")[1].strip()
        return response.strip()
    
A_chain = answer_prompt | llm | AnswerParser()


In [7]:
# 5 단계 : 768d 를 자르고 임베딩한 후 여러 방법으로 합치기
def Accuracy_test(documents, embed_model, model_name, chunk_size):
    
    # 1. Langchain의 RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)

    # 모델 불러오기 (HuggingFaceEmbeddings 사용)
    # embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')

    # 문서 예시 (documents_512에서 첫 번째 문서)
    for doc in documents:
        
        text = doc.page_content
        metadata = doc.metadata
        # 2. 임베딩 벡터와 관련된 기존 데이터 초기화
        for key in list(metadata.keys()):
            if 'embedding_vector' in key or 'num_chunks' in key:
                del metadata[key]  # 이전 임베딩 관련 데이터 삭제
                
        # 2. RecursiveCharacterTextSplitter로 문서를 128토큰 단위로 분할
        split_documents = splitter.split_text(text)

        # 3. 각 분할된 텍스트를 임베딩
        chunk_embeddings = [embed_model.embed_query(chunk) for chunk in split_documents]

##################################combined_embedding 한개만 쓰면됨################################################################

        # 4. 768 차원으로 결합하는 방식 (method에 따라 다름)
        if model_name == "768_Mean":
            combined_embedding = np.mean(chunk_embeddings, axis=0)
        elif model_name == "768_First":
            combined_embedding = np.concatenate([chunk[:768 // len(chunk_embeddings)] for chunk in chunk_embeddings], axis=0)
        elif model_name == "768_Each":
            num_chunks = len(chunk_embeddings)
            split_size = 768 // num_chunks  # 각 청크에서 가져올 크기 계산
            combined_embedding = []

            # 각 청크에서 순차적으로 부분을 가져오는 방식
            for i, chunk in enumerate(chunk_embeddings):
                start_index = i * split_size
                end_index = (i + 1) * split_size

                # 각 청크에서 해당 부분을 결합
                combined_embedding.append(chunk[start_index:end_index])

            combined_embedding = np.concatenate(combined_embedding, axis=0)

            # 결합된 벡터가 768차원이 안 되면 마지막 청크에서 부족한 부분을 가져와 채우기
            if combined_embedding.shape[0] < 768:
                remaining_size = 768 - combined_embedding.shape[0]
                combined_embedding = np.concatenate([combined_embedding, chunk_embeddings[-1][:remaining_size]], axis=0)

        else:
            raise ValueError("Invalid method specified.")

        # Metadata에 벡터 저장
        metadata[f'{model_name}embedding_vector'] = combined_embedding.tolist()
        metadata['num_chunks'] = len(chunk_embeddings)

        # print(f"Document ID: {metadata['id']} - Embedding created with {model_name}")


##################################combined_embedding 한개만 쓰면됨################################################################

    # 문서를 딕셔너리로 변환하여 JSON으로 저장
    documents = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

    with open(f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_{chunk_size}_results.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=4)

In [8]:
# 5 단계 : 768d 를 자르고 임베딩한 후 concatenation 하기
def Accuracy_test(documents, embed_model, model_name, chunk_size):
    
    # 1. Langchain의 RecursiveCharacterTextSplitter 사용
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)

    # 모델 불러오기 (HuggingFaceEmbeddings 사용)
    # embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')

    # 문서 예시 (documents_512에서 첫 번째 문서)
    for doc in documents:
        
        text = doc.page_content
        metadata = doc.metadata
        # 2. 임베딩 벡터와 관련된 기존 데이터 초기화
        for key in list(metadata.keys()):
            if 'embedding_vector' in key or 'num_chunks' in key:
                del metadata[key]  # 이전 임베딩 관련 데이터 삭제
                
        # 2. RecursiveCharacterTextSplitter로 문서를 128토큰 단위로 분할
        split_documents = splitter.split_text(text)

        # 3. 각 분할된 텍스트를 임베딩
        chunk_embeddings = [embed_model.embed_query(chunk) for chunk in split_documents]

##################################combined_embedding 한개만 쓰면됨################################################################

        # 4. 768 차원으로 결합하는 방식 (method에 따라 다름)
        if model_name == "768_Mean":
            combined_embedding = np.mean(chunk_embeddings, axis=0)
            
        elif model_name == "768_First":
            combined_embedding = np.concatenate([chunk[:768 // len(chunk_embeddings)] for chunk in chunk_embeddings], axis=0)
            if combined_embedding.shape[0] < 768:
                remaining_size = 768 - combined_embedding.shape[0]
                combined_embedding = np.concatenate([combined_embedding, chunk_embeddings[-1][:remaining_size]], axis=0)
                
        elif model_name == "768_Each":
            num_chunks = len(chunk_embeddings)
            split_size = 768 // num_chunks  # 각 청크에서 가져올 크기 계산
            combined_embedding = []

            # 각 청크에서 순차적으로 부분을 가져오는 방식
            for i, chunk in enumerate(chunk_embeddings):
                start_index = i * split_size
                end_index = (i + 1) * split_size

                # 각 청크에서 해당 부분을 결합
                combined_embedding.append(chunk[start_index:end_index])

            combined_embedding = np.concatenate(combined_embedding, axis=0)

            # 결합된 벡터가 768차원이 안 되면 마지막 청크에서 부족한 부분을 가져와 채우기
            if combined_embedding.shape[0] < 768:
                remaining_size = 768 - combined_embedding.shape[0]
                combined_embedding = np.concatenate([combined_embedding, chunk_embeddings[-1][:remaining_size]], axis=0)

        else:
            raise ValueError("Invalid method specified.")

        # Metadata에 벡터 저장
        metadata[f'{model_name}embedding_vector'] = combined_embedding.tolist()
        metadata['num_chunks'] = len(chunk_embeddings)

        # print(f"Document ID: {metadata['id']} - Embedding created with {model_name}")


##################################combined_embedding 한개만 쓰면됨################################################################

    # 문서를 딕셔너리로 변환하여 JSON으로 저장
    documents = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

    with open(f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_{chunk_size}_results.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=4)

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name='/home/choi/Git/RAG_con_doc/langchain/FT_model/MRL_MNRL_NLI-2024-10-17_14-16-30/checkpoint/checkpoint-322_best')
embedding_models2 = [
    # ('EM_klue_nli', EM_klue_nli),
    (embed_model, '768_Mean'),
    (embed_model, '768_First'),
    (embed_model, '768_Each'),
]
for embed_model, model_name in embedding_models2:
    Accuracy_test(documents_512, embed_model, model_name, 128)
# Accuracy_test(documents_512, embed_model, "768_Mean")

In [15]:
import numpy as np
import json
from sklearn.metrics import ndcg_score

# 문서 검색 및 정확도 계산 함수
def search_documents(query, embed_model, model_name, documents):
    query_embedding = embed_model.embed_query(query)
    distances = []

    for doc in documents:
        doc_vector = np.array(doc.metadata[f'{model_name}embedding_vector'])  # Document의 First 임베딩 사용
        # num_chunks = doc.metadata['num_chunks']  # 해당 문서의 청크 수

        # # 쿼리 임베딩의 앞부분만 사용 (문서의 청크 수에 맞춰 자르기)
        # split_size = 768 // num_chunks
        # query_front_part = query_embedding[:split_size]  # 쿼리 임베딩의 앞부분만 사용

        # 각 문서 청크와 쿼리의 앞부분을 비교
        chunk_distances = []
        # for i in range(num_chunks):
        #     doc_chunk = doc_vector[i * split_size:(i + 1) * split_size]  # 문서의 청크 임베딩
        #     distance = np.linalg.norm(query_front_part - doc_chunk) * num_chunks**0.5  # L2 거리 계산
        #     chunk_distances.append(distance)
        distance = np.linalg.norm(query_embedding - doc_vector) ** 0.5
        # 각 청크의 거리 중 최소값을 해당 문서의 거리로 사용
        # min_distance = distance
        distances.append((doc, distance))
    
    # 거리 기준으로 가장 가까운 문서 5개 찾기
    distances.sort(key=lambda x: x[1])  # 거리가 작은 순서대로 정렬
    top_5 = distances[:5]  # 상위 5개의 문서
    
    # 상위 5개 문서 ID 추출
    top_5_ids = [doc.metadata['id'] for doc, _ in top_5]
    
    # 상위 검색 문서의 첫 번째 문서를 반환
    top_document = top_5[0][0]  # 첫 번째 문서 객체 반환
    return top_5_ids, top_document

# 전체 문서 검색 및 답변 생성
def evaluate_documents(embed_model, model_name, documents, A_chain):
    results = []
    correct_count = 0  # 정답을 맞춘 문서의 개수를 세기 위한 변수
    ndcg_scores = []
    mrr_scores = []

    for doc in documents:
        question = doc.metadata['question']  # 각 문서의 질문을 쿼리로 사용
        correct_id = doc.metadata['id']  # 정답 ID
        correct_answer = doc.metadata['answer']  # 정답 답변

        # 해당 문서의 question을 쿼리로 사용하여 문서 검색
        retrieved_ids, top_document = search_documents(question, embed_model, model_name, documents)

        # 답변 생성 (가장 유사한 문서를 사용하여 답변 생성)
        # generated_answer = A_chain.invoke({
        #     'question': question,
        #     'input': top_document.page_content  # 가장 유사한 문서의 내용을 사용하여 답변 생성
        # })

        # NDCG와 MRR 계산을 위한 true 값 및 score
        y_true = [1 if retrieved_id == correct_id else 0 for retrieved_id in retrieved_ids]
        y_score = [5, 4, 3, 2, 1]  # 순위에 따른 가중치

        # NDCG@5 계산
        ndcg = ndcg_score([y_true], [y_score], k=5)
        ndcg_scores.append(ndcg)

        # MRR@5 계산
        try:
            rank = retrieved_ids.index(correct_id) + 1
            mrr = 1 / rank
        except ValueError:
            mrr = 0  # 정답이 top 5 안에 없으면 MRR은 0
        mrr_scores.append(mrr)

        # 결과 저장
        results.append({
            'question': question,
            'correct_id': correct_id,
            'retrieved_ids': retrieved_ids,
            'correct_answer': correct_answer,
            # 'generated_answer': generated_answer,
            'ndcg@5': ndcg,
            'mrr@5': mrr
        })

    # 전체 NDCG와 MRR 평균 계산
    avg_ndcg = sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0
    avg_mrr = sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0

    print(f"평균 NDCG@5: {avg_ndcg:.4f}")
    print(f"평균 MRR@5: {avg_mrr:.4f}")

    return results, avg_ndcg, avg_mrr

# JSON 파일에서 데이터를 불러와 Document 객체로 변환하는 함수
def load_documents_from_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # JSON 데이터를 Document 객체로 변환
    documents = [
        Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in data
    ]
    return documents

# 저장된 파일 경로
model_name = '768_Each'
chunk_size = '128'
filepath = f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_{chunk_size}_results.json'

# 문서를 불러와 Document 객체로 변환
documents_test = load_documents_from_json(filepath)

print(f"불러온 문서의 수: {len(documents_test)}")

# 각 문서의 질문을 쿼리로 사용하여 검색 및 답변 생성 평가
results, avg_ndcg, avg_mrr = evaluate_documents(embed_model, model_name, documents_test, A_chain)

# 결과를 JSON 파일로 저장
with open(f'/home/choi/Git/RAG_con_doc/langchain/{model_name}_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# 저장된 결과 확인
for result in results:
    print(result)

불러온 문서의 수: 1255
평균 NDCG@5: 0.5067
평균 MRR@5: 0.4646
{'question': '태양광 발전 설비 공사에서 옥상 안전난간대 설치 시, 건축공사분의 옥상 안전난간대와 겹칠 경우 어떻게 조치해야 하나요?', 'correct_id': '4a61ae9d-43c7-42ce-8104-39bcf66c25be', 'retrieved_ids': ['4a61ae9d-43c7-42ce-8104-39bcf66c25be', '9f2c0426-0869-46e1-9cdd-b2c37ddee5a1', '8d34c62f-1538-4db4-945e-9c195b016e89', 'eb2771f4-0389-480c-8663-785236618262', '08c3c293-b28c-40eb-b6e2-57fc70be773e'], 'correct_answer': '건축공사분의 옥상 안전난간대와 겹칠 경우, 건축공사분의 옥상 안전난간대를 설계변경 조치하고, 태양전지판이 설치되는 면만 안전난간대를 시공해야 합니다.', 'ndcg@5': 1.0, 'mrr@5': 1.0}
{'question': '태양광 발전설비 공사에서, 수급인은 계약 체결일로부터 며칠 이내에 설치공정표와 안전관리 계획서를 감독자에게 제출하여야 합니까?', 'correct_id': '82bb0b35-8f9f-480e-abb7-9caba32757a9', 'retrieved_ids': ['f38ca29d-3d00-4050-b20e-58557fcbc77b', '8e99d86b-bc84-4071-8326-59841ae5d531', '82bb0b35-8f9f-480e-abb7-9caba32757a9', '8f5d64f6-f690-4968-a2de-ab30d04b65a4', 'eb2771f4-0389-480c-8663-785236618262'], 'correct_answer': '수급인은 계약 체결일로부터 30일 이내에 설치공정표와 안전관리 계획서를 감독자에게 제출하여야 합니다.', 'ndcg@5': 0.5, 'mrr@5

In [15]:
# JSON 파일에서 결과를 불러오는 함수
def load_results(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# NDCG@5가 1점인 항목 개수를 계산하는 함수
def count_perfect_ndcg(results):
    perfect_ndcg_count = 0

    for result in results:
        if result['ndcg@5'] == 1.0:
            perfect_ndcg_count += 1

    return perfect_ndcg_count

# 평균 NDCG@5와 MRR@5 계산 함수
def calculate_avg_scores(results):
    ndcg_scores = []
    mrr_scores = []

    for result in results:
        ndcg_scores.append(result['ndcg@5'])
        mrr_scores.append(result['mrr@5'])

    # 평균 계산
    avg_ndcg = sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0
    avg_mrr = sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0

    return avg_ndcg, avg_mrr

# JSON 파일 경로
filepath = '/home/choi/Git/RAG_con_doc/langchain/768_First_128_results.json'

# JSON 파일에서 결과 불러오기
results = load_results(filepath)

# 평균 NDCG@5와 MRR@5 계산
avg_ndcg, avg_mrr = calculate_avg_scores(results)
# NDCG@5가 1점인 항목 개수 계산
perfect_ndcg_count = count_perfect_ndcg(results)

# 결과 출력
print(f"NDCG@5가 1점인 항목의 개수: {perfect_ndcg_count}")

# 결과 출력
print(f"평균 NDCG@5: {avg_ndcg:.4f}")
print(f"평균 MRR@5: {avg_mrr:.4f}")


KeyError: 'ndcg@5'